In [1]:
import pandas as pd
from util import list_contains, count_item

In [2]:
def create_headertable(data, min_support_ratio:float):
    c_item_set = []
    origin_itemset_valcount = data.item.value_counts()
    for i in data.item:
        new_elems = list(i)
        c_item_set = c_item_set + list((set(new_elems) - set(c_item_set)))
    
    df = pd.DataFrame({
        "itemset": c_item_set,
        "support": count_item(origin_itemset_valcount, c_item_set, init=True)
    })
    
    df.drop(df[df.support < min_support_ratio].index, inplace=True) # prune

    return df

In [3]:
odf = pd.read_csv("../../data/ibm.csv")
df = odf.groupby(['transaction_id']).item_id.apply(tuple).reset_index(name='item')
df["length"] = df.item.apply(lambda x: len(x))
len(df)

828

In [4]:
header_table = create_headertable(df, 0.1)
# header_table = header_table.set_index('itemset').T.to_dict('list')
header_table

,itemset,support
0,307,0.138889
1,723,0.141304
2,470,0.212560
3,443,0.190821
4,973,0.158213
5,3,0.410628
6,451,0.404589
7,647,0.108696
8,488,0.403382
9,487,0.404589


In [5]:
class Node(object):
    def __init__(self, item, count, parent=None):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.next = None
        if parent is not None:
            parent.children[item] = self
    def display(self, ind=1):
        print('  ' *  ind, (self.item), ' ', str(self.count) )
        for child in list(self.children.values()):
            child.display(ind+1)

class FPTree(object):
    def __init__(self):
        super().__init__()
        self.root = Node(None, 1)
    
    def insert_itemsets(self, itemsets, count=1):
        self.root.count += count
        if len(itemsets) == 0:
            return 
        
        cur_node = self.root
        for item in itemsets:
            try:
                child = cur_node.children[item]
                child.count += count
                cur_node = child
            except:
                new_node = Node(item, 1, cur_node)
                cur_node.children[item] = new_node                
                cur_node = new_node
                # update header
                if (header_table[item][1] == None):
                    header_table[item][1] = new_node
                else:
                    up_node = header_table[item][1]
                    while up_node.next != None:
                        up_node = up_node.next
                    up_node.next = new_node

def _ascend_fp_tree(node, prefix_path: list):
    if node.parent != None:
        prefix_path.append(node.item)
        _ascend_fp_tree(node.parent, prefix_path)

def _find_prefix_path(base_pat, header_table):
    treeNode = header_table[base_pat][1] 
    condPats = []
    frequency = []
    while treeNode != None:
        prefixPath = []
        # From leaf node all the way to root
        _ascend_fp_tree(treeNode, prefixPath)  
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            condPats.append(prefixPath[1:])
            frequency.append(treeNode.count)

        # Go to next node
        treeNode = treeNode.next  
    return condPats, frequency



                
        

In [6]:
tree = FPTree()
for itemsets in df.item.tolist():
    itemsets = [item for item in itemsets if item in header_table]
    itemsets.sort(key=lambda item: header_table[item][0], reverse=True)
    tree.insert_itemsets(itemsets, 1)

In [7]:
tree.root.display()

   None   829
